In [89]:
client_id = "gv6WrVUFyZviqGBAtlSV"
client_secret = "nJ3Kxb67QL"

In [93]:
import requests
import pandas as pd

def naver_api(query, search_type="blog", display=10):
    # API URL
    # 블로그-> blog, 뉴스-> news, 책-> book, 백과사전->encyc, 
    # 카페글->cafearticle, 지식인->kin, 웹문서->webkr, 이미지->image, 
    # 쇼핑->shop, 전문자료->doc, 성인검색어 판별->adult, 오타변환->errata
    url = f"https://openapi.naver.com/v1/search/{search_type}.json"
    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }
    params = {
    'query': query,
    'display': display,}

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error 발생:", response.status_code)
        return None
    
data = []
def make_df(query, search_type) : 
    result = naver_api(query, search_type)
    if result:
        for item in result['items']:
            row = {
                "title": item.get("title", "").replace("<b>", "").replace("</b>", ""),
                "link": item.get("link", ""),
                "description": item.get("description", "").replace("<b>", "").replace("</b>", "")
            }
            for key in item:
                if key not in row:
                    row[key] = item[key]
            data.append(row)
    return pd.DataFrame(data)

In [ ]:
body > table.type2 > tbody > tr:nth-child(14)
body > table.type2 > tbody > tr:nth-child(15)body > table.type2 > tbody > tr:nth-child(3)
body > table.type2 > tbody > tr:nth-child(2)

In [ ]:
APP_KEY = "YOUR_APP_KEY"
APP_SECRET = "YOUR_APP_SECRET"
ACCESS_TOKEN = "YOUR_ACCESS_TOKEN"  # 토큰을 미리 받아와야 함
TR_ID = "HHISQ031"  # 주식 기간별 시세 조회 API

In [ ]:

import requests
import pandas as pd
from datetime import datetime

# 주식 기간별 데이터 수집 함수
def get_stock_history(stock_code, start_date, end_date):
    url = "https://openapi.koreainvestment.com:9443/uapi/domestic-stock/v1/quotations/inquire-daily-price"
    headers = {
        "Content-Type": "application/json",
        "authorization": f"Bearer {ACCESS_TOKEN}",
        "appkey": APP_KEY,
        "appsecret": APP_SECRET,
        "tr_id": TR_ID
    }
    params = {
        "FID_COND_MRKT_DIV_CODE": "J",  # 코스피: J, 코스닥: Q
        "FID_INPUT_ISCD": stock_code,  # 종목 코드 (예: 삼성전자 "005930")
        "FID_INPUT_DATE_1": start_date,  # 조회 시작일 (YYYYMMDD)
        "FID_INPUT_DATE_2": end_date,  # 조회 종료일 (YYYYMMDD)
        "FID_PERIOD_DIV_CODE": "D",  # 일봉: D, 주봉: W, 월봉: M
        "FID_ORG_ADJ_PRC": "0"  # 수정 주가 여부 (0: 수정 주가, 1: 미조정)
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if "output" in data:
            return data["output"]
        else:
            print("No data found")
            return None
    else:
        print("API 요청 실패:", response.status_code, response.text)
        return None

# DataFrame 변환 함수
def stock_data_to_dataframe(stock_data):
    if not stock_data:
        return None
    
    df = pd.DataFrame(stock_data)
    
    # 컬럼 이름 변경
    df = df.rename(columns={
        "stck_bsop_date": "date",
        "stck_oprc": "open",
        "stck_hgpr": "high",
        "stck_lwpr": "low",
        "stck_clpr": "close",
        "acml_vol": "volume"
    })
    
    # 데이터 타입 변환
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
    df["open"] = df["open"].astype(float)
    df["high"] = df["high"].astype(float)
    df["low"] = df["low"].astype(float)
    df["close"] = df["close"].astype(float)
    df["volume"] = df["volume"].astype(int)
    
    # 날짜 기준 정렬 (내림차순 -> 오름차순)
    df = df.sort_values(by="date").reset_index(drop=True)
    
    return df

# 실행 예제
if __name__ == "__main__":
    stock_code = "005930"  # 삼성전자
    start_date = "20240101"  # 시작일 (YYYYMMDD)
    end_date = "20240310"  # 종료일 (YYYYMMDD)

    stock_data = get_stock_history(stock_code, start_date, end_date)
    df = stock_data_to_dataframe(stock_data)

    if df is not None:
        print(df.head())  # 데이터 확인
